In [2]:
## This is for capturing the images from the cam and safe it at location
## capture images with c and quit program with q


# Spinnaker SDK
import os
from pyspin import PySpin as spin
import sys
import keyboard
import cv2

# coding=utf-8
# =============================================================================
# Copyright (c) 2001-2021 FLIR Systems, Inc. All Rights Reserved.
#
# This software is the confidential and proprietary information of FLIR
# Integrated Imaging Solutions, Inc. ("Confidential Information"). You
# shall not disclose such Confidential Information and shall use it only in
# accordance with the terms of the license agreement you entered into
# with FLIR Integrated Imaging Solutions, Inc. (FLIR).
#
# FLIR MAKES NO REPRESENTATIONS OR WARRANTIES ABOUT THE SUITABILITY OF THE
# SOFTWARE, EITHER EXPRESSED OR IMPLIED, INCLUDING, BUT NOT LIMITED TO, THE
# IMPLIED WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR
# PURPOSE, OR NON-INFRINGEMENT. FLIR SHALL NOT BE LIABLE FOR ANY DAMAGES
# SUFFERED BY LICENSEE AS A RESULT OF USING, MODIFYING OR DISTRIBUTING
# THIS SOFTWARE OR ITS DERIVATIVES.
# =============================================================================
#
# Acquisition.py shows how to acquire images. It relies on
# information provided in the Enumeration example. Also, check out the
# ExceptionHandling and NodeMapInfo examples if you haven't already.
# ExceptionHandling shows the handling of standard and Spinnaker exceptions
# while NodeMapInfo explores retrieving information from various node types.
#
# This example touches on the preparation and cleanup of a camera just before
# and just after the acquisition of images. Image retrieval and conversion,
# grabbing image data, and saving images are all covered as well.
#
# Once comfortable with Acquisition, we suggest checking out
# AcquisitionMultipleCamera, NodeMapCallback, or SaveToAvi.
# AcquisitionMultipleCamera demonstrates simultaneously acquiring images from
# a number of cameras, NodeMapCallback serves as a good introduction to
# programming with callbacks and events, and SaveToAvi exhibits video creation.




## parameters

calibPictureFilename = "Test"
ROI_y = 1227
ROI_x = 584
ROI_height = 800
ROI_width = 800



def acquire_images(cam, nodemap, nodemap_tldevice):
    """
    This function acquires and saves 10 images from a device.

    :param cam: Camera to acquire images from.
    :param nodemap: Device nodemap.
    :param nodemap_tldevice: Transport layer device nodemap.
    :type cam: CameraPtr
    :type nodemap: INodeMap
    :type nodemap_tldevice: INodeMap
    :return: True if successful, False otherwise.
    :rtype: bool
    """

    print('*** IMAGE ACQUISITION ***\n')
    try:
        result = True

        # Set acquisition mode to continuous
        #
        #  *** NOTES ***
        #  Because the example acquires and saves 10 images, setting acquisition
        #  mode to continuous lets the example finish. If set to single frame
        #  or multiframe (at a lower number of images), the example would just
        #  hang. This would happen because the example has been written to
        #  acquire 10 images while the camera would have been programmed to
        #  retrieve less than that.
        #
        #  Setting the value of an enumeration node is slightly more complicated
        #  than other node types. Two nodes must be retrieved: first, the
        #  enumeration node is retrieved from the nodemap; and second, the entry
        #  node is retrieved from the enumeration node. The integer value of the
        #  entry node is then set as the new value of the enumeration node.
        #
        #  Notice that both the enumeration and the entry nodes are checked for
        #  availability and readability/writability. Enumeration nodes are
        #  generally readable and writable whereas their entry nodes are only
        #  ever readable.
        #
        #  Retrieve enumeration node from nodemap

        # In order to access the node entries, they have to be casted to a pointer type (CEnumerationPtr here)
        node_acquisition_mode = spin.CEnumerationPtr(nodemap.GetNode('AcquisitionMode'))
        if not spin.IsAvailable(node_acquisition_mode) or not spin.IsWritable(node_acquisition_mode):
            print('Unable to set acquisition mode to continuous (enum retrieval). Aborting...')
            return False

        # Retrieve entry node from enumeration node
        node_acquisition_mode_continuous = node_acquisition_mode.GetEntryByName('Continuous')
        if not spin.IsAvailable(node_acquisition_mode_continuous) or not spin.IsReadable(node_acquisition_mode_continuous):
            print('Unable to set acquisition mode to continuous (entry retrieval). Aborting...')
            return False

        # Retrieve integer value from entry node
        acquisition_mode_continuous = node_acquisition_mode_continuous.GetValue()

        # Set integer value from entry node as new value of enumeration node
        node_acquisition_mode.SetIntValue(acquisition_mode_continuous)

        print('Acquisition mode set to continuous...')

        #  Begin acquiring images
        #
        #  *** NOTES ***
        #  What happens when the camera begins acquiring images depends on the
        #  acquisition mode. Single frame captures only a single image, multi
        #  frame catures a set number of images, and continuous captures a
        #  continuous stream of images. Because the example calls for the
        #  retrieval of 10 images, continuous mode has been set.
        #
        #  *** LATER ***
        #  Image acquisition must be ended when no more images are needed.
        #cam.PixelFormat.SetValue(spin.PixelFormat_BGR8)
        cam.BeginAcquisition()

        print('Acquiring images...')

        #  Retrieve device serial number for filename
        #
        #  *** NOTES ***
        #  The device serial number is retrieved in order to keep cameras from
        #  overwriting one another. Grabbing image IDs could also accomplish
        #  this.
        device_serial_number = ''
        node_device_serial_number = spin.CStringPtr(nodemap_tldevice.GetNode('DeviceSerialNumber'))
        if spin.IsAvailable(node_device_serial_number) and spin.IsReadable(node_device_serial_number):
            device_serial_number = node_device_serial_number.GetValue()
            print('Device serial number retrieved as %s...' % device_serial_number)

        # Retrieve, convert, and save images
        ## for continous tream: change this into while true?
        i = 0
        while not keyboard.is_pressed('q'):
            try:
                if keyboard.is_pressed('c'):  # if key 'c' is pressed 
                    print('You Pressed A Key!')
                    i = i + 1
    
    
                

                #  Retrieve next received image
                #
                #  *** NOTES ***
                #  Capturing an image houses images on the camera buffer. Trying
                #  to capture an image that does not exist will hang the camera.
                #
                #  *** LATER ***
                #  Once an image from the buffer is saved and/or no longer
                #  needed, the image must be released in order to keep the
                #  buffer from filling up.
                    image_result = cam.GetNextImage(1000)

                #  Ensure image completion
                #
                #  *** NOTES ***
                #  Images can easily be checked for completion. This should be
                #  done whenever a complete image is expected or required.
                #  Further, check image status for a little more insight into
                #  why an image is incomplete.
                    if image_result.IsIncomplete():
                        print('Image incomplete with image status %d ...' % image_result.GetImageStatus())

                    else:

                    #  Print image information; height and width recorded in pixels
                    #
                    #  *** NOTES ***
                    #  Images have quite a bit of available metadata including
                    #  things such as CRC, image status, and offset values, to
                    #  name a few.
                        width = image_result.GetWidth()
                        height = image_result.GetHeight()
                        print('Grabbed Image %d, width = %d, height = %d' % (i, width, height))

                    #  Convert image to mono 8
                    #
                    #  *** NOTES ***
                    #  Images can be converted between pixel formats by using
                    #  the appropriate enumeration value. Unlike the original
                    #  image, the converted one does not need to be released as
                    #  it does not affect the camera buffer.
                    #
                    #  When converting images, color processing algorithm is an
                    #  optional parameter.
                    
                    ## This converts it to GreyScale
                    #image_converted = image_result.Convert(spin.PixelFormat_Mono8, spin.HQ_LINEAR)
                    ## This converts it to RGB
                        #image_converted = image_result.Convert(spin.PixelFormat_BGR8)
                        
                        
                        image_converted = image_result.Convert(spin.PixelFormat_BGR8)
                        rgb_array = image_converted.GetData()
                        rgb_array = rgb_array.reshape(height, width, 3)
                    
                            
                        ## process mediapipe on image
                        image_rgb = cv2.cvtColor(cv2.flip(rgb_array, 1), cv2.COLOR_BGR2RGB)
                            
                            
                        ## **** Resizing / Croping *****
                            
                        ## RESIZING the image since it would be 2048x2048 otherwise (kind of too big for the window)
                        #image_rgb = cv2.resize(image_rgb, (800, 800))
                        ## this is to display potential cropping region in the downscaled image
                        # scale = 800/2048   
                        # cv2.rectangle(image_rgb, (int(650*scale), int(580*scale)), (int(1450*scale), int(1380*scale)), (0, 255, 0), 3)
                                                        
                        ## CROPPPING the region of the lense (should be around 800 to fit with the setup so far...)
                        ## This is the cropping
                        ## These values are taken from the unity config
                        ## They might change... [y:y+h, x:x+h]
                        array_cropped = image_rgb[ROI_y:(ROI_y + ROI_height), ROI_x:(ROI_x + ROI_width)]
                        image_rgb = array_cropped.copy() # needed to get the correct data format for further processing
                          
                        image_rgb = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2BGR)
                    
                    
                    
                    
                        ## Saving the image to a folder
                        # Create a unique filename
                        if calibPictureFilename:
                            filename = 'CameraFeed\%s-%d.jpg' % (calibPictureFilename, i)
                        else:
                            if device_serial_number:
                                filename = 'CameraFeed\Acquisition-%s-%d.jpg' % (device_serial_number, i)
                            else:  # if serial number is empty
                                filename = 'CameraFeed\Acquisition-%d.jpg' % i
                               
                        #  Save image
                        #
                        #  *** NOTES ***
                        #  The standard practice of the examples is to use device
                        #  serial numbers to keep images of one device from
                        #  overwriting those of another.
                        #image_converted.Save(filename)
                        cv2.imwrite(filename, image_rgb)
                        print('Image saved at %s' % filename)

                        #  Release image
                        #
                        #  *** NOTES ***
                        #  Images retrieved directly from the camera (i.e. non-converted
                        #  images) need to be released in order to keep from filling the
                        #  buffer.
                        image_result.Release()
                        print('')

                
                else:
                    ## when no capturing key is pressed the image still needs to be loaded, otherwise the images will pile up in the buffer
                    image_result = cam.GetNextImage(1000)
                    image_result.Release()
                
                
                
                
            except spin.SpinnakerException as ex:
                print('Error: %s' % ex)
                return False

        #  End acquisition
        #
        #  *** NOTES ***
        #  Ending acquisition appropriately helps ensure that devices clean up
        #  properly and do not need to be power-cycled to maintain integrity.
        cam.EndAcquisition()
    
    except spin.SpinnakerException as ex:
        print('Error: %s' % ex)
        return False

    return result






def print_device_info(nodemap):
    """
    This function prints the device information of the camera from the transport
    layer; please see NodeMapInfo example for more in-depth comments on printing
    device information from the nodemap.

    :param nodemap: Transport layer device nodemap.
    :type nodemap: INodeMap
    :returns: True if successful, False otherwise.
    :rtype: bool
    """

    print('*** DEVICE INFORMATION ***\n')

    try:
        result = True
        node_device_information = spin.CCategoryPtr(nodemap.GetNode('DeviceInformation'))

        if spin.IsAvailable(node_device_information) and spin.IsReadable(node_device_information):
            features = node_device_information.GetFeatures()
            for feature in features:
                node_feature = spin.CValuePtr(feature)
                print('%s: %s' % (node_feature.GetName(),
                                  node_feature.ToString() if spin.IsReadable(node_feature) else 'Node not readable'))

        else:
            print('Device control information not available.')

    except spin.SpinnakerException as ex:
        print('Error: %s' % ex)
        return False

    return result







def run_single_camera(cam):
    """
    This function acts as the body of the example; please see NodeMapInfo example
    for more in-depth comments on setting up cameras.

    :param cam: Camera to run on.
    :type cam: CameraPtr
    :return: True if successful, False otherwise.
    :rtype: bool
    """
    try:
        result = True

        # Retrieve TL device nodemap and print device information
        nodemap_tldevice = cam.GetTLDeviceNodeMap()

        result &= print_device_info(nodemap_tldevice)

        # Initialize camera
        cam.Init()

        # Retrieve GenICam nodemap
        nodemap = cam.GetNodeMap()

        # Acquire images
        result &= acquire_images(cam, nodemap, nodemap_tldevice)

        # Deinitialize camera
        cam.DeInit()

    except spin.SpinnakerException as ex:
        print('Error: %s' % ex)
        result = False

    return result










def main():
    """
    Example entry point; please see Enumeration example for more in-depth
    comments on preparing and cleaning up the system.

    :return: True if successful, False otherwise.
    :rtype: bool
    """

    # Since this application saves images in the current folder
    # we must ensure that we have permission to write to this folder.
    # If we do not have permission, fail right away.
    try:
        test_file = open('test.txt', 'w+')
    except IOError:
        print('Unable to write to current directory. Please check permissions.')
        input('Press Enter to exit...')
        return False

    test_file.close()
    os.remove(test_file.name)

    result = True

    # Retrieve singleton reference to system object
    system = spin.System.GetInstance()

    # Get current library version
    version = system.GetLibraryVersion()
    print('Library version: %d.%d.%d.%d' % (version.major, version.minor, version.type, version.build))

    # Retrieve list of cameras from the system
    cam_list = system.GetCameras()

    num_cameras = cam_list.GetSize()

    print('Number of cameras detected: %d' % num_cameras)

    # Finish if there are no cameras
    if num_cameras == 0:

        # Clear camera list before releasing system
        cam_list.Clear()

        # Release system instance
        system.ReleaseInstance()

        print('Not enough cameras!')
        input('Done! Press Enter to exit...')
        return False

    # Run example on each camera
    for i, cam in enumerate(cam_list):

        print('Running example for camera %d...' % i)

        result &= run_single_camera(cam)
        print('Camera %d example complete... \n' % i)

    # Release reference to camera
    # NOTE: Unlike the C++ examples, we cannot rely on pointer objects being automatically
    # cleaned up when going out of scope.
    # The usage of del is preferred to assigning the variable to None.
    del cam

    # Clear camera list before releasing system
    cam_list.Clear()

    # Release system instance
    system.ReleaseInstance()

    input('Done! Press Enter to exit...')
    return result

if __name__ == '__main__':
    if main():
        sys.exit(0)
    else:
        sys.exit(1)


Library version: 2.4.0.144
Number of cameras detected: 1
Running example for camera 0...
*** DEVICE INFORMATION ***

DeviceID: USB\VID_1E10&PID_3300&MI_00\6&284BB4E7&1&0000
DeviceSerialNumber: 20413825
DeviceVendorName: Point Grey Research
DeviceModelName: Grasshopper3 GS3-U3-41C6NIR
DeviceType: USB3Vision
DeviceDisplayName: Point Grey Research Grasshopper3 GS3-U3-41C6NIR
DeviceAccessStatus: OpenReadWrite
DeviceVersion: FW:v2.23.3.02 FPGA:v2.02
DeviceDriverVersion: PGRUSBCam.sys : 2.7.3.235
DeviceUserID: 
DeviceIsUpdater: 0
DeviceInstanceId: USB\VID_1E10&PID_3300&MI_00\6&284BB4E7&1&0000
DeviceLocation: 0000.0014.0000.020.000.000.000.000.000
DeviceCurrentSpeed: SuperSpeed
GUIXMLLocation: Device
GUIXMLPath: Input.xml
GenICamXMLLocation: Device
GenICamXMLPath: 
DeviceU3VProtocol: 1
DevicePortId: 20
*** IMAGE ACQUISITION ***

Acquisition mode set to continuous...
Acquiring images...
Device serial number retrieved as 20413825...
You Pressed A Key!
Grabbed Image 1, width = 2048, height = 204

You Pressed A Key!
Grabbed Image 71, width = 2048, height = 2048
Image saved at CameraFeed\Test-71.jpg

You Pressed A Key!
Grabbed Image 72, width = 2048, height = 2048
Image saved at CameraFeed\Test-72.jpg

You Pressed A Key!
Grabbed Image 73, width = 2048, height = 2048
Image saved at CameraFeed\Test-73.jpg

You Pressed A Key!
Grabbed Image 74, width = 2048, height = 2048
Image saved at CameraFeed\Test-74.jpg

You Pressed A Key!
Grabbed Image 75, width = 2048, height = 2048
Image saved at CameraFeed\Test-75.jpg

You Pressed A Key!
Grabbed Image 76, width = 2048, height = 2048
Image saved at CameraFeed\Test-76.jpg

You Pressed A Key!
Grabbed Image 77, width = 2048, height = 2048
Image saved at CameraFeed\Test-77.jpg

You Pressed A Key!
Grabbed Image 78, width = 2048, height = 2048
Image saved at CameraFeed\Test-78.jpg

You Pressed A Key!
Grabbed Image 79, width = 2048, height = 2048
Image saved at CameraFeed\Test-79.jpg

You Pressed A Key!
Grabbed Image 80, width = 2048, height = 2048

You Pressed A Key!
Grabbed Image 150, width = 2048, height = 2048
Image saved at CameraFeed\Test-150.jpg

You Pressed A Key!
Grabbed Image 151, width = 2048, height = 2048
Image saved at CameraFeed\Test-151.jpg

You Pressed A Key!
Grabbed Image 152, width = 2048, height = 2048
Image saved at CameraFeed\Test-152.jpg

You Pressed A Key!
Grabbed Image 153, width = 2048, height = 2048
Image saved at CameraFeed\Test-153.jpg

You Pressed A Key!
Grabbed Image 154, width = 2048, height = 2048
Image saved at CameraFeed\Test-154.jpg

You Pressed A Key!
Grabbed Image 155, width = 2048, height = 2048
Image saved at CameraFeed\Test-155.jpg

You Pressed A Key!
Grabbed Image 156, width = 2048, height = 2048
Image saved at CameraFeed\Test-156.jpg

You Pressed A Key!
Grabbed Image 157, width = 2048, height = 2048
Image saved at CameraFeed\Test-157.jpg

You Pressed A Key!
Grabbed Image 158, width = 2048, height = 2048
Image saved at CameraFeed\Test-158.jpg

You Pressed A Key!
Grabbed Image 159, width = 

SystemExit: 0

In [3]:
## OLD: Undistort for normal lense but we need to use fisheye calibration


import numpy as np
import cv2 as cv
import glob
import keyboard
import time


## parameters

calibPictureFilename = "TestFeed\Test"





def main():
    # termination criteria
    criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)

    # prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
    objp = np.zeros((6*9,3), np.float32)
    objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

    # Arrays to store object points and image points from all the images.
    objpoints = [] # 3d point in real world space
    imgpoints = [] # 2d points in image plane.

    images = glob.glob('TestFeed/*.jpg')

    i=0

    for fname in images:
    
        img = cv.imread(fname)
        print('Search image in %s' %fname)
    
        gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    
        # Find the chess board corners
        ret, corners = cv.findChessboardCorners(gray, (9,6), None)
    
        # If found, add object points, image points (after refining them)
        if ret == True:
            objpoints.append(objp)
            corners2 = cv.cornerSubPix(gray,corners, (11,11), (-1,-1), criteria)
            imgpoints.append(corners)
        
            # Draw and display the corners
            cv.drawChessboardCorners(img, (9,6), corners2, ret)
            cv.imshow('img', img)
            cv.imwrite('%s-%d.jpg' %(calibPictureFilename, i), img)
            print('Image saved at %s' % fname)
            cv.waitKey(500)
        
        
            ## camera calibration
            ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
            
            print(fname)
            img = cv.imread(fname)
            print(img)
            h,  w = img.shape[:2]
            newcameramtx, roi = cv.getOptimalNewCameraMatrix(mtx, dist, (w,h), 1, (w,h))
            
            # undistort
            dst = cv.undistort(img, mtx, dist, None, newcameramtx)
            # crop the image
            x, y, w, h = roi
            dst = dst[y:y+h, x:x+w]
            cv.imwrite('TestFeed\calibresult_%d.png' %i , dst)
            
            ## this stores the objectpoints in a textfile for now bccccccccccccccccqut the format does not work yet
            file = open("TestFeed/Test.txt", "a")
            file.write("objectPoints, " + str(objpoints))
            
            # NEXT: undistort picture
            
        
            
            i = i + 1
        
        else:
            print('No chessboard found')
 


    cv.destroyAllWindows()
    

    
    

main()

Search image in TestFeed\Test-10.jpg
Image saved at TestFeed\Test-10.jpg
TestFeed\Test-10.jpg
[[[33 24 21]
  [34 25 22]
  [36 24 24]
  ...
  [36 24 22]
  [38 25 23]
  [39 26 24]]

 [[34 25 22]
  [34 25 22]
  [36 24 24]
  ...
  [36 24 22]
  [38 25 23]
  [38 25 23]]

 [[34 25 22]
  [34 25 22]
  [36 24 24]
  ...
  [36 24 22]
  [37 25 23]
  [37 25 23]]

 ...

 [[34 25 22]
  [34 25 22]
  [33 24 21]
  ...
  [34 24 24]
  [37 25 23]
  [38 26 24]]

 [[32 23 20]
  [33 24 21]
  [33 24 21]
  ...
  [35 23 21]
  [37 24 22]
  [37 24 22]]

 [[31 22 19]
  [33 24 21]
  [34 25 22]
  ...
  [35 23 21]
  [35 22 20]
  [34 21 19]]]
Search image in TestFeed\Test-11.jpg
Image saved at TestFeed\Test-11.jpg
TestFeed\Test-11.jpg
[[[38 25 23]
  [38 25 23]
  [37 25 23]
  ...
  [36 23 21]
  [38 24 25]
  [41 27 28]]

 [[38 25 23]
  [37 24 22]
  [36 24 22]
  ...
  [36 23 21]
  [38 24 25]
  [39 25 26]]

 [[37 24 22]
  [37 24 22]
  [35 23 21]
  ...
  [36 24 24]
  [36 24 24]
  [36 24 24]]

 ...

 [[36 24 22]
  [33 24 21]


In [6]:
## fisheye camera calibration
## use this to get the parameters and copy them to the undistortion function

import cv2
import numpy as np
import os
import glob

CHECKERBOARD = (5,7)

subpix_criteria = (cv2.TERM_CRITERIA_EPS+cv2.TERM_CRITERIA_MAX_ITER, 30, 0.1)
calibration_flags = cv2.fisheye.CALIB_RECOMPUTE_EXTRINSIC+cv2.fisheye.CALIB_CHECK_COND+cv2.fisheye.CALIB_FIX_SKEW

objp = np.zeros((1, CHECKERBOARD[0]*CHECKERBOARD[1], 3), np.float32)
objp[0,:,:2] = np.mgrid[0:CHECKERBOARD[0], 0:CHECKERBOARD[1]].T.reshape(-1, 2)

_img_shape = None

objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.

images = glob.glob('CameraFeed\*.jpg')

for fname in images:
    print("Processing image " + str(fname))
    img = cv2.imread(fname)
    if _img_shape == None:
        _img_shape = img.shape[:2]
    else:
        assert _img_shape == img.shape[:2], "All images must share the same size."
        

    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    # Find the chess board corners
    ret, corners = cv2.findChessboardCorners(gray, CHECKERBOARD, cv2.CALIB_CB_ADAPTIVE_THRESH+cv2.CALIB_CB_FAST_CHECK+cv2.CALIB_CB_NORMALIZE_IMAGE)
    # If found, add object points, image points (after refining them)
    if ret == True:
        objpoints.append(objp)
        cv2.cornerSubPix(gray,corners,(3,3),(-1,-1),subpix_criteria)
        imgpoints.append(corners)
        print("Valid image: " + str(fname))
        
N_OK = len(objpoints)
K = np.zeros((3, 3))
D = np.zeros((4, 1))
rvecs = [np.zeros((1, 1, 3), dtype=np.float64) for i in range(N_OK)]
tvecs = [np.zeros((1, 1, 3), dtype=np.float64) for i in range(N_OK)]
rms, _, _, _, _ = \
    cv2.fisheye.calibrate(
        objpoints,
        imgpoints,
        gray.shape[::-1],
        K,
        D,
        rvecs,
        tvecs,
        calibration_flags,
        (cv2.TERM_CRITERIA_EPS+cv2.TERM_CRITERIA_MAX_ITER, 30, 1e-6)
    )

print("Found " + str(N_OK) + " valid images for calibration")
print("DIM=" + str(_img_shape[::-1]))
print("K=np.array(" + str(K.tolist()) + ")")
print("D=np.array(" + str(D.tolist()) + ")")


Processing image CameraFeed\Test-10.jpg
Valid image: CameraFeed\Test-10.jpg
Processing image CameraFeed\Test-100.jpg
Valid image: CameraFeed\Test-100.jpg
Processing image CameraFeed\Test-101.jpg
Valid image: CameraFeed\Test-101.jpg
Processing image CameraFeed\Test-102.jpg
Valid image: CameraFeed\Test-102.jpg
Processing image CameraFeed\Test-103.jpg
Valid image: CameraFeed\Test-103.jpg
Processing image CameraFeed\Test-104.jpg
Valid image: CameraFeed\Test-104.jpg
Processing image CameraFeed\Test-105.jpg
Valid image: CameraFeed\Test-105.jpg
Processing image CameraFeed\Test-106.jpg
Valid image: CameraFeed\Test-106.jpg
Processing image CameraFeed\Test-107.jpg
Valid image: CameraFeed\Test-107.jpg
Processing image CameraFeed\Test-108.jpg
Valid image: CameraFeed\Test-108.jpg
Processing image CameraFeed\Test-109.jpg
Processing image CameraFeed\Test-112.jpg
Valid image: CameraFeed\Test-112.jpg
Processing image CameraFeed\Test-113.jpg
Valid image: CameraFeed\Test-113.jpg
Processing image CameraFe

Processing image CameraFeed\Test-65.jpg
Processing image CameraFeed\Test-66.jpg
Processing image CameraFeed\Test-67.jpg
Processing image CameraFeed\Test-68.jpg
Processing image CameraFeed\Test-7.jpg
Valid image: CameraFeed\Test-7.jpg
Processing image CameraFeed\Test-75.jpg
Valid image: CameraFeed\Test-75.jpg
Processing image CameraFeed\Test-76.jpg
Processing image CameraFeed\Test-77.jpg
Processing image CameraFeed\Test-78.jpg
Valid image: CameraFeed\Test-78.jpg
Processing image CameraFeed\Test-79.jpg
Valid image: CameraFeed\Test-79.jpg
Processing image CameraFeed\Test-8.jpg
Valid image: CameraFeed\Test-8.jpg
Processing image CameraFeed\Test-80.jpg
Valid image: CameraFeed\Test-80.jpg
Processing image CameraFeed\Test-81.jpg
Valid image: CameraFeed\Test-81.jpg
Processing image CameraFeed\Test-82.jpg
Valid image: CameraFeed\Test-82.jpg
Processing image CameraFeed\Test-86.jpg
Processing image CameraFeed\Test-87.jpg
Processing image CameraFeed\Test-88.jpg
Processing image CameraFeed\Test-89.

In [4]:
### undistort with parameters from fisheye calibration

# insert the output of fisheye calibration step
DIM=(800, 800)
K=np.array([[351.6318033232932, 0.0, 450.4607527123814], [0.0, 351.7439651791754, 391.2576486450391], [0.0, 0.0, 1.0]])
D=np.array([[-0.170592708935433], [0.5324235902617314], [-1.5452235955907878], [1.4793950832426657]])




def undistort(img_path, balance=0.0, dim2=None, dim3=None):
    
    img = cv2.imread(img_path)
    
    dim1 = img.shape[:2][::-1]  #dim1 is the dimension of input image to un-distort
    
   # assert dim1[0]/dim1[1] == DIM[0]/DIM[1], "Image to undistort needs to have same aspect ratio as the ones used in calibration"
    
    if not dim2:
        dim2 = dim1
    if not dim3:
        dim3 = dim1
        
        
    scaled_K = K * dim1[0] / DIM[0]  # The values of K is to scale with image dimension.
    scaled_K[2][2] = 1.0  # Except that K[2][2] is always 1.0
    
    # This is how scaled_K, dim2 and balance are used to determine the final K used to un-distort image. OpenCV document failed to make this clear!
    new_K = cv2.fisheye.estimateNewCameraMatrixForUndistortRectify(scaled_K, D, dim2, np.eye(3), balance=balance)
    map1, map2 = cv2.fisheye.initUndistortRectifyMap(scaled_K, D, np.eye(3), new_K, dim3, cv2.CV_16SC2)
    undistorted_img = cv2.remap(img, map1, map2, interpolation=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT)
    
    cv2.imshow("distorted", img)
    cv2.imshow("undistorted", undistorted_img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
    
    
    
    
if __name__ == '__main__':
    undistort('TestFeed\Test-12.jpg', 1.0, (800,800), (800, 800))
    #for p in sys.argv[1:]:
    #    undistort(p)
    